In [90]:
import numpy as np
import psycopg2
from psycopg2 import OperationalError
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.bool_, psycopg2._psycopg.AsIs)
from pytrends.request import TrendReq
from time import sleep
import pandas as pd
import seaborn as sns
import json

pytrends = TrendReq(hl='en-US', tz=360)

In [91]:
#get credentials
with open("postgre_credentials.json") as file:
    data = json.load(file)

In [92]:
def reset_connection():
    if 'connection' not in locals():
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection,cur;
    else:
        connection.close()
        connection = psycopg2.connect(
        database = data["db_name"], 
        user = data["db_user"],
        password = data["db_password"],
        host = data["db_host"],
        port = data["db_port"]
        )
        cur = connection.cursor()
        return connection, cur;

In [93]:
connection, cur = reset_connection()

In [94]:
query="""SELECT eng_name, alternate_name
FROM anime join show_name ON anime.anime_id = show_name.anime_id
WHERE air_status LIKE 'Finished Airing'
"""
cur.execute(query)
anime_ids = cur.fetchall()

In [96]:
#anime_ids

In [97]:
#input country string - Example: 'US'
#input show_tuple - tuple with eng_name as the first entry. May contain 'NA'
#output - google trends_table is filled with show info with show_name attached
def call_google_trends(country, show_tuple):    
    try:
        name = "fill"
        for entry in show_tuple:
            if entry != "NA":
                name = entry
                break       
        if name!= "fill":
            pytrends.build_payload(kw_list=[name],cat=0,timeframe='all', geo=country)
            data = pytrends.interest_over_time()
            #print(data)
            if not data.empty:
                data = data.drop(labels=['isPartial'],axis='columns')
                return data
    except (RuntimeError):
        print("Invalid Name?")
        print(RuntimeError)

In [98]:
def insert_pop_score(show_df):
    #NEED TO FIRST GET show_name_id through query, and then input
#     for row in show_df.iterrows():
#         popularity_score = row[1][0]
#         date_id = row[1][1]
#         show_name = YOU HAVE TO GET THIS IN THE QUERY
#         cur.execute("""INSERT INTO google_trends(show_name, date, popularity_score)
#                     VALUES (%s, %s, %s)
#                     """,(show_name, date_id, popularity_score))
#    connection.commit()

In [124]:

anime_ids = [entry for entry in anime_ids if entry[0]!='NA']

In [126]:
anime_ids[16]

('Death Billiards', 'デス・ビリヤード')

In [127]:
test_df = call_google_trends('US', anime_ids[16])

In [128]:
insert_pop_score(test_df)

,Death Billiards
date,
2004-01-01,0
2004-02-01,0
2004-03-01,0
2004-04-01,0
2004-05-01,0
...,...
2020-08-01,2
2020-09-01,2
2020-10-01,6


In [129]:
connection.close()